In [4]:
import os
import pandas as pd
seed = 3423

In [5]:
def remove_duplicates(path, columns):
    df = pd.read_csv(path)
    df = df.drop(columns=columns)
    df = df.dropna()
    df = df.drop_duplicates()
    df, _ = strip_all_duplicates(df)
    print(len(df), len(pd.read_csv(path)) - len(df))
    return df

def strip_all_duplicates(df):
    r = pd.DataFrame()
    for column in ['title', 'text']:
        c = df[column]
        flag = c.isin(c[c.duplicated()])
        r = pd.concat([r, df.loc[flag]])
    r = r.drop_duplicates()
    return df[~df.index.isin(r.index)], r

def concat_and_shuffle(dfs):
    global seed
    df = pd.concat(dfs)
    df = df.sample(frac=1, random_state=seed)
    seed += 1
    return df

def relabel(label):
    if label in ['REAL', 'FAKE']:
        return label
    if label == 0:
        return 'REAL'
    elif label == 1:
        return 'FAKE'
    # this shouldn't be an issue for these two datasets
    warnings.warn(f"Unexpected label {label}. Setting equal to 'UNK'.")
    return -1

In [6]:
# get data and remove duplicates
INPUTS = '../inputs/'

PATH1 = INPUTS + 'news.csv'
if not os.path.exists(PATH1):
    !unzip news.csv.zip -d {INPUTS}
df1 = remove_duplicates(PATH1, ['Unnamed: 0'])

PATH2 = INPUTS + 'train.csv'
if not os.path.exists(PATH2):
    !unzip fake-news.zip -d {INPUTS}
df2 = remove_duplicates(PATH2, ['id', 'author'])

# combine data and make labels uniform
df = concat_and_shuffle([df1, df2])
df['label'] = df.label.apply(relabel)

df.to_csv(INPUTS + 'news_data_groomed.csv')

5923 412
19136 1664
